<a href="https://colab.research.google.com/github/machine-learning-red/recsys/blob/main/an_recsys_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# INSTALAMOS EL MODULO QUE IMPLEMENTA EL ALGORITMO Apriori
!pip install apriori_python

In [33]:
# IMPORTAMOS EL MODULO Y CREAMOS NUESTRO DATASET DE PRUEBAS
import csv
import sys
import pandas as pd
from google.colab import drive
from apriori_python import apriori
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#####
# IMPORTAMOS EL ARCHIVO Y REEMPLAZAMOS LOS CEROS POR VALOR NULO
#####
url = "/content/drive/MyDrive/MachineLearning/ProyectoFinal/dataset_SP_LIV_Exclusivo.csv"
df = pd.read_csv(url)
apriori = df.copy()
for column in df:
    if column != "ID_Cliente":
      apriori.loc[(df[column] > 0), column] = column
      apriori.loc[(df[column] == 0), column] = None
apriori.head()

,JUV,VIS,DOG,AD,INT,TRI,VIE,DOE,DOB,KOC,DBE,KAZ,DPT,MSG,KON,BIM,DOA,LEG,DOP,DOV,CPT,AH,PLU,PRA,CON,THA,ZEN,MAG,KRJ,MOP,DPO,MSA,TER,KRS,VEN,DOF,PUF,MOC,X4,X5,...,DAL,SUR,SIB,MAL,ULT,BIL,GEN,TOT,HAM,TOR,KOB,SEV,GOA,TOL,HUA,THU,EST,RAV,SIE,SPM,PRO,200K,LBA,HAG,MED,LUC,GRE,NAV,ECO,SPA,SUP,EMU,CDB,PT,DOM,NOR,HAR,JAK,HOL,POC
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,AH,PLU,PRA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,VIE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,PLU,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,KOC,None,None,DPT,None,None,None,None,None,None,None,None,None,None,PRA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,VIS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,PLU,PRA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,TRI,VIE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CON,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [31]:
#####
# ABRIMOS EL ARCHIVO, FORMATEAMOS Y REEMPLAZAMOS EL FORMATO POR EL UTILIZADO POR APRIORI
#####
DS_APRIORI_FILENAME = '/content/drive/MyDrive/MachineLearning/ProyectoFinal/dataset_SP_LIV_Exclusivo.csv'
f = open(DS_APRIORI_FILENAME, 'w') # open the file in the write mode
writer = csv.writer(f) # create the csv writer
csv = StringIO(u""+apriori.to_csv(index=False, header=False))
for row in csv.readlines():
  r = row
  try:
    while r.index(",,"):
      r = r.replace(",,",",")
  except ValueError:
    pass # ignore not finding any more commas
  r = r[:-1] # removes \n and the last comma
  res = r.split(",")[1:]
  if res[0] == "":
    continue
  else:
    writer.writerow(res) # write a row to the csv file
f.close() # close the file

In [34]:
# CREAMOS UN ARRAY CON TODAS LAS TRANSACCIONES Y CALCULAMOS ALGUNAS ESTADISTICAS BASICAS
from io import StringIO
transactions = []
min = sys.maxsize
max = 0
mean = 0
acc = 0
with open('/content/drive/MyDrive/MachineLearning/ProyectoFinal/dataset_SP_LIV_Exclusivo.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      row = row[:-1]
      transactions.append(row)
      size = len(row)
      acc += size
      if size < min:
        min = len(row)
      if size > max:
        max = len(row)

print("#######################################")
print("ESTADISTICAS DE LA BD DE TRANSACCIONES:")
print("#######################################")  
print(f"NUMERO TOTAL DE TRANSACCIONES: {len(transactions)}")
print(f"ITEMSET DE MENOR TAMAÑO......: {min}")
print(f"ITEMSET DE MAYOR TAMAÑO......: {max}")
print(f"TAMAÑO MEDIO DE ITEMSETS.....: {acc/len(transactions)}")

#######################################
ESTADISTICAS DE LA BD DE TRANSACCIONES:
#######################################
NUMERO TOTAL DE TRANSACCIONES: 733
ITEMSET DE MENOR TAMAÑO......: 1
ITEMSET DE MAYOR TAMAÑO......: 14
TAMAÑO MEDIO DE ITEMSETS.....: 4.466575716234652


In [88]:
freqItemSet, rules = apriori(transactions, minSup=0.1, minConf=0.8)

In [89]:
print("####################")
print("ITEMSETS FRECUENTES:")
print("####################")
for key in freqItemSet.keys():
  print(f"ITEMSETS FRECUENTES DE TAMAÑO {key}:")
  print("  " + str(freqItemSet[key]))


####################
ITEMSETS FRECUENTES:
####################
ITEMSETS FRECUENTES DE TAMAÑO 1:
  {frozenset({'DBE'}), frozenset({'PLU'}), frozenset({'CPT'}), frozenset({'VIE'}), frozenset({'CON'}), frozenset({'PRA'}), frozenset({'VIS'}), frozenset({'PUF'}), frozenset({'INT'}), frozenset({'KOC'}), frozenset({'DPT'})}
ITEMSETS FRECUENTES DE TAMAÑO 2:
  {frozenset({'DPT', 'VIS'}), frozenset({'DBE', 'VIS'}), frozenset({'INT', 'VIS'}), frozenset({'VIE', 'VIS'}), frozenset({'PRA', 'VIS'}), frozenset({'CON', 'VIS'}), frozenset({'VIS', 'PLU'}), frozenset({'PUF', 'VIS'})}


In [90]:
print("#####################")
print("REGLAS DE ASOCIACION:")
print("#####################")
for item in rules:
  print(item)

#####################
REGLAS DE ASOCIACION:
#####################
[{'PRA'}, {'VIS'}, 0.8091603053435115]
[{'INT'}, {'VIS'}, 0.8102189781021898]
[{'PUF'}, {'VIS'}, 0.8134328358208955]
[{'DPT'}, {'VIS'}, 0.8235294117647058]
[{'VIE'}, {'VIS'}, 0.8255813953488372]
[{'PLU'}, {'VIS'}, 0.8681318681318682]
[{'DBE'}, {'VIS'}, 0.8813559322033898]
